In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use("ggplot")
from sklearn.preprocessing import MinMaxScaler
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dfopenmeteo = pd.read_csv("drive/MyDrive/DatasetsDATATHON26/open_meteo.csv")

In [ ]:
dfopenmeteo.head(10)

,location_id,time,temperature_2m (°C),cloud_cover (%),wind_speed_100m (km/h),shortwave_radiation (W/m²),apparent_temperature (°C),direct_radiation (W/m²),diffuse_radiation (W/m²),wind_gusts_10m (km/h),relative_humidity_2m (%)
0,0,2024-01-01T00:00,3.2,100,13.3,0.0,0.6,0.0,0.0,9.4,93
1,0,2024-01-01T01:00,4.0,100,16.3,0.0,1.1,0.0,0.0,10.8,81
2,0,2024-01-01T02:00,4.0,100,14.0,0.0,1.2,0.0,0.0,10.8,76
3,0,2024-01-01T03:00,3.8,100,12.3,0.0,1.0,0.0,0.0,9.4,77
4,0,2024-01-01T04:00,3.3,100,13.1,0.0,0.5,0.0,0.0,9.0,80
5,0,2024-01-01T05:00,3.8,100,15.3,0.0,0.9,0.0,0.0,10.4,78
6,0,2024-01-01T06:00,4.3,100,19.1,0.0,1.0,0.0,0.0,15.5,75
7,0,2024-01-01T07:00,4.2,100,21.1,0.0,0.6,0.0,0.0,18.0,74
8,0,2024-01-01T08:00,4.3,100,21.1,0.0,0.7,0.0,0.0,18.4,72
9,0,2024-01-01T09:00,3.7,85,20.7,2.0,0.2,0.0,2.0,18.4,75


In [ ]:
dfopenmeteo.dtypes

,0
location_id,int64
time,object
temperature_2m (°C),float64
cloud_cover (%),int64
wind_speed_100m (km/h),float64
shortwave_radiation (W/m²),float64
apparent_temperature (°C),float64
direct_radiation (W/m²),float64
diffuse_radiation (W/m²),float64
wind_gusts_10m (km/h),float64


In [ ]:
dfopenmeteo.describe()

,location_id,temperature_2m (°C),cloud_cover (%),wind_speed_100m (km/h),shortwave_radiation (W/m²),apparent_temperature (°C),direct_radiation (W/m²),diffuse_radiation (W/m²),wind_gusts_10m (km/h),relative_humidity_2m (%)
count,412848.000000,412848.000000,412848.000000,412848.000000,412848.000000,412848.000000,412848.000000,412848.000000,412848.000000,412848.000000
mean,23.000000,15.569751,51.055253,16.509947,189.137000,14.129560,130.317814,58.819185,22.635843,68.711569
std,13.564676,7.782810,42.625378,10.515938,268.116753,8.950844,213.412246,78.574825,13.025709,21.542565
min,0.000000,-13.200000,0.000000,0.000000,0.000000,-18.400000,0.000000,0.000000,0.700000,5.000000
25%,11.000000,10.000000,3.000000,8.200000,0.000000,7.500000,0.000000,0.000000,12.200000,54.000000
50%,23.000000,15.100000,49.000000,14.800000,9.000000,13.500000,0.000000,7.000000,20.200000,73.000000
75%,35.000000,20.600000,100.000000,22.800000,342.000000,20.200000,198.000000,106.000000,30.200000,87.000000
max,46.000000,43.500000,100.000000,90.100000,1017.000000,43.800000,899.000000,476.000000,120.600000,100.000000


In [ ]:
numeric_cols = dfopenmeteo.select_dtypes(include=[np.number]).columns
if 'location_id' in numeric_cols:
    numeric_cols = numeric_cols.drop('location_id')

dfopenmeteo_grouped = dfopenmeteo.groupby('time')[numeric_cols].mean().reset_index()

# Timezone handling
dfopenmeteo_grouped['time'] = pd.to_datetime(dfopenmeteo_grouped['time']).dt.tz_localize('UTC')
dfopenmeteo_grouped = dfopenmeteo_grouped[:-1]
dfopenmeteo_grouped

,time,temperature_2m (°C),cloud_cover (%),wind_speed_100m (km/h),shortwave_radiation (W/m²),apparent_temperature (°C),direct_radiation (W/m²),diffuse_radiation (W/m²),wind_gusts_10m (km/h),relative_humidity_2m (%)
0,2024-01-01 00:00:00+00:00,6.376596,96.808511,13.372340,0.000000,3.921277,0.000000,0.000000,13.587234,84.574468
1,2024-01-01 01:00:00+00:00,6.185106,99.255319,13.114894,0.000000,3.742553,0.000000,0.000000,13.136170,85.000000
2,2024-01-01 02:00:00+00:00,6.457447,99.723404,12.072340,0.000000,4.055319,0.000000,0.000000,12.563830,82.872340
3,2024-01-01 03:00:00+00:00,6.327660,96.574468,11.227660,0.000000,4.006383,0.000000,0.000000,11.759574,83.382979
4,2024-01-01 04:00:00+00:00,5.991489,95.744681,11.391489,0.000000,3.589362,0.000000,0.000000,11.563830,84.574468
...,...,...,...,...,...,...,...,...,...,...
8778,2024-12-31 18:00:00+00:00,5.989362,12.425532,9.380851,40.404255,3.759574,23.127660,17.276596,11.306383,78.531915
8779,2024-12-31 19:00:00+00:00,4.889362,13.085106,9.095745,0.170213,2.582979,0.042553,0.127660,9.078723,82.489362
8780,2024-12-31 20:00:00+00:00,4.068085,14.191489,9.221277,0.000000,1.631915,0.000000,0.000000,9.612766,84.723404
8781,2024-12-31 21:00:00+00:00,3.325532,17.659574,9.985106,0.000000,0.712766,0.000000,0.000000,10.676596,86.127660


In [ ]:
dfpvpc = pd.read_csv("drive/MyDrive/DatasetsDATATHON26/export_TerminoDeFacturacionDeEnergiaActivaDelPVPC20TD_2026-01-22_23_39.csv", sep=';')

# Standardize columns
# We only need datetime and value
dfpvpc = dfpvpc.rename(columns={'datetime': 'time', 'value': 'price'})
dfpvpc = dfpvpc[['time', 'price']]

# Timezone handling
# PVPC data is usually in ISO format with offsets. Convert to UTC.
dfpvpc['time'] = pd.to_datetime(dfpvpc['time'], utc=True)

# Sort
dfpvpc = dfpvpc.sort_values('time')


dfpvpc = dfpvpc[1:]
dfpvpc

,time,price
1,2024-01-01 00:00:00+00:00,111.11
2,2024-01-01 01:00:00+00:00,110.77
3,2024-01-01 02:00:00+00:00,109.28
4,2024-01-01 03:00:00+00:00,110.16
5,2024-01-01 04:00:00+00:00,111.47
...,...,...
8779,2024-12-31 18:00:00+00:00,274.90
8780,2024-12-31 19:00:00+00:00,271.27
8781,2024-12-31 20:00:00+00:00,268.41
8782,2024-12-31 21:00:00+00:00,212.84


In [ ]:
df = pd.merge(dfpvpc, dfopenmeteo_grouped, on='time', how='inner')
df = df.sort_values('time').reset_index(drop=True)
df

,time,price,temperature_2m (°C),cloud_cover (%),wind_speed_100m (km/h),shortwave_radiation (W/m²),apparent_temperature (°C),direct_radiation (W/m²),diffuse_radiation (W/m²),wind_gusts_10m (km/h),relative_humidity_2m (%)
0,2024-01-01 00:00:00+00:00,111.11,6.376596,96.808511,13.372340,0.000000,3.921277,0.000000,0.000000,13.587234,84.574468
1,2024-01-01 01:00:00+00:00,110.77,6.185106,99.255319,13.114894,0.000000,3.742553,0.000000,0.000000,13.136170,85.000000
2,2024-01-01 02:00:00+00:00,109.28,6.457447,99.723404,12.072340,0.000000,4.055319,0.000000,0.000000,12.563830,82.872340
3,2024-01-01 03:00:00+00:00,110.16,6.327660,96.574468,11.227660,0.000000,4.006383,0.000000,0.000000,11.759574,83.382979
4,2024-01-01 04:00:00+00:00,111.47,5.991489,95.744681,11.391489,0.000000,3.589362,0.000000,0.000000,11.563830,84.574468
...,...,...,...,...,...,...,...,...,...,...,...
8778,2024-12-31 18:00:00+00:00,274.90,5.989362,12.425532,9.380851,40.404255,3.759574,23.127660,17.276596,11.306383,78.531915
8779,2024-12-31 19:00:00+00:00,271.27,4.889362,13.085106,9.095745,0.170213,2.582979,0.042553,0.127660,9.078723,82.489362
8780,2024-12-31 20:00:00+00:00,268.41,4.068085,14.191489,9.221277,0.000000,1.631915,0.000000,0.000000,9.612766,84.723404
8781,2024-12-31 21:00:00+00:00,212.84,3.325532,17.659574,9.985106,0.000000,0.712766,0.000000,0.000000,10.676596,86.127660


In [ ]:
df.isna().sum()

,0
time,0
price,0
temperature_2m (°C),0
cloud_cover (%),0
wind_speed_100m (km/h),0
shortwave_radiation (W/m²),0
apparent_temperature (°C),0
direct_radiation (W/m²),0
diffuse_radiation (W/m²),0
wind_gusts_10m (km/h),0


In [ ]:
df.loc[df.iloc[:,[1,2,3,4,5,6,7,8,9,10]].duplicated()]

,time,price,temperature_2m (°C),cloud_cover (%),wind_speed_100m (km/h),shortwave_radiation (W/m²),apparent_temperature (°C),direct_radiation (W/m²),diffuse_radiation (W/m²),wind_gusts_10m (km/h),relative_humidity_2m (%)


In [ ]:
# 3. Extraer variables básicas de tiempo
df['hour'] = df['time'].dt.hour
df['day_of_week'] = df['time'].dt.dayofweek  # 0=Lunes, 6=Domingo
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['month'] = df['time'].dt.month

# 4. Definir festivos nacionales de España (2024)
festivos_2024 = [
    '2024-01-01', '2024-01-06', '2024-03-29', '2024-05-01',
    '2024-08-15', '2024-10-12', '2024-11-01', '2024-12-06', '2024-12-25'
]
df['is_holiday'] = df['time'].dt.strftime('%Y-%m-%d').isin(festivos_2024).astype(int)

# 5. Transformación Cíclica (Evita que el modelo crea que la hora 23 es "lejana" a la 0)
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)


In [ ]:

df["no_laborable"]=df[["is_weekend","is_holiday"]].sum(axis=1)


In [ ]:
df["no_laborable"]=df["no_laborable"].replace(2,1)

In [ ]:
df

,time,price,temperature_2m (°C),cloud_cover (%),wind_speed_100m (km/h),shortwave_radiation (W/m²),apparent_temperature (°C),direct_radiation (W/m²),diffuse_radiation (W/m²),wind_gusts_10m (km/h),...,is_weekend,month,is_holiday,hour_sin,hour_cos,day_sin,day_cos,month_sin,month_cos,no_laborable
0,2024-01-01 00:00:00+00:00,111.11,6.376596,96.808511,13.372340,0.000000,3.921277,0.000000,0.000000,13.587234,...,0,1,1,0.000000,1.000000e+00,0.000000,1.00000,5.000000e-01,0.866025,1
1,2024-01-01 01:00:00+00:00,110.77,6.185106,99.255319,13.114894,0.000000,3.742553,0.000000,0.000000,13.136170,...,0,1,1,0.258819,9.659258e-01,0.000000,1.00000,5.000000e-01,0.866025,1
2,2024-01-01 02:00:00+00:00,109.28,6.457447,99.723404,12.072340,0.000000,4.055319,0.000000,0.000000,12.563830,...,0,1,1,0.500000,8.660254e-01,0.000000,1.00000,5.000000e-01,0.866025,1
3,2024-01-01 03:00:00+00:00,110.16,6.327660,96.574468,11.227660,0.000000,4.006383,0.000000,0.000000,11.759574,...,0,1,1,0.707107,7.071068e-01,0.000000,1.00000,5.000000e-01,0.866025,1
4,2024-01-01 04:00:00+00:00,111.47,5.991489,95.744681,11.391489,0.000000,3.589362,0.000000,0.000000,11.563830,...,0,1,1,0.866025,5.000000e-01,0.000000,1.00000,5.000000e-01,0.866025,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8778,2024-12-31 18:00:00+00:00,274.90,5.989362,12.425532,9.380851,40.404255,3.759574,23.127660,17.276596,11.306383,...,0,12,0,-1.000000,-1.836970e-16,0.781831,0.62349,-2.449294e-16,1.000000,0
8779,2024-12-31 19:00:00+00:00,271.27,4.889362,13.085106,9.095745,0.170213,2.582979,0.042553,0.127660,9.078723,...,0,12,0,-0.965926,2.588190e-01,0.781831,0.62349,-2.449294e-16,1.000000,0
8780,2024-12-31 20:00:00+00:00,268.41,4.068085,14.191489,9.221277,0.000000,1.631915,0.000000,0.000000,9.612766,...,0,12,0,-0.866025,5.000000e-01,0.781831,0.62349,-2.449294e-16,1.000000,0
8781,2024-12-31 21:00:00+00:00,212.84,3.325532,17.659574,9.985106,0.000000,0.712766,0.000000,0.000000,10.676596,...,0,12,0,-0.707107,7.071068e-01,0.781831,0.62349,-2.449294e-16,1.000000,0


In [ ]:
diasnolaborables2024=df[df["no_laborable"]==1].shape[0]/24
print(diasnolaborables2024)

111.0


In [ ]:
df[(df["is_weekend"]!=df['no_laborable']) & (df["is_holiday"]!=df['no_laborable'])]

,time,price,temperature_2m (°C),cloud_cover (%),wind_speed_100m (km/h),shortwave_radiation (W/m²),apparent_temperature (°C),direct_radiation (W/m²),diffuse_radiation (W/m²),wind_gusts_10m (km/h),...,is_weekend,month,is_holiday,hour_sin,hour_cos,day_sin,day_cos,month_sin,month_cos,no_laborable


In [ ]:
df.drop("is_weekend",axis=1,inplace=True)
df.drop("is_holiday",axis=1,inplace=True)
df.drop("day_of_week",axis=1,inplace=True)
df.drop("hour",axis=1,inplace=True)
df.drop("time",axis=1,inplace=True)
df.drop('month', axis=1, inplace=True)

In [ ]:
df

,price,temperature_2m (°C),cloud_cover (%),wind_speed_100m (km/h),shortwave_radiation (W/m²),apparent_temperature (°C),direct_radiation (W/m²),diffuse_radiation (W/m²),wind_gusts_10m (km/h),relative_humidity_2m (%),hour_sin,hour_cos,day_sin,day_cos,month_sin,month_cos,no_laborable
0,111.11,6.376596,96.808511,13.372340,0.000000,3.921277,0.000000,0.000000,13.587234,84.574468,0.000000,1.000000e+00,0.000000,1.00000,5.000000e-01,0.866025,1
1,110.77,6.185106,99.255319,13.114894,0.000000,3.742553,0.000000,0.000000,13.136170,85.000000,0.258819,9.659258e-01,0.000000,1.00000,5.000000e-01,0.866025,1
2,109.28,6.457447,99.723404,12.072340,0.000000,4.055319,0.000000,0.000000,12.563830,82.872340,0.500000,8.660254e-01,0.000000,1.00000,5.000000e-01,0.866025,1
3,110.16,6.327660,96.574468,11.227660,0.000000,4.006383,0.000000,0.000000,11.759574,83.382979,0.707107,7.071068e-01,0.000000,1.00000,5.000000e-01,0.866025,1
4,111.47,5.991489,95.744681,11.391489,0.000000,3.589362,0.000000,0.000000,11.563830,84.574468,0.866025,5.000000e-01,0.000000,1.00000,5.000000e-01,0.866025,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8778,274.90,5.989362,12.425532,9.380851,40.404255,3.759574,23.127660,17.276596,11.306383,78.531915,-1.000000,-1.836970e-16,0.781831,0.62349,-2.449294e-16,1.000000,0
8779,271.27,4.889362,13.085106,9.095745,0.170213,2.582979,0.042553,0.127660,9.078723,82.489362,-0.965926,2.588190e-01,0.781831,0.62349,-2.449294e-16,1.000000,0
8780,268.41,4.068085,14.191489,9.221277,0.000000,1.631915,0.000000,0.000000,9.612766,84.723404,-0.866025,5.000000e-01,0.781831,0.62349,-2.449294e-16,1.000000,0
8781,212.84,3.325532,17.659574,9.985106,0.000000,0.712766,0.000000,0.000000,10.676596,86.127660,-0.707107,7.071068e-01,0.781831,0.62349,-2.449294e-16,1.000000,0


#Escalado de todas las variables

In [ ]:
# 2. Separar el objetivo (precio) de las características
target_col = 'price'
feature_cols = [col for col in df.columns if col != target_col]

# 3. Inicializar los escaladores
# Es vital escalar el precio por separado para poder des-normalizar las predicciones después
scaler_features = MinMaxScaler(feature_range=(0, 1))
scaler_target = MinMaxScaler(feature_range=(0, 1))

# 4. Ajustar y transformar los datos
df_scaled = pd.DataFrame(scaler_features.fit_transform(df[feature_cols]), columns=feature_cols)
df_scaled[target_col] = scaler_target.fit_transform(df[[target_col]])

# Reordenar para que el precio vuelva a ser la primera columna (opcional)
df_scaled = df_scaled[[target_col] + feature_cols]


In [ ]:
df_scaled

,price,temperature_2m (°C),cloud_cover (%),wind_speed_100m (km/h),shortwave_radiation (W/m²),apparent_temperature (°C),direct_radiation (W/m²),diffuse_radiation (W/m²),wind_gusts_10m (km/h),relative_humidity_2m (%),hour_sin,hour_cos,day_sin,day_cos,month_sin,month_cos,no_laborable
0,0.306377,0.153884,0.968085,0.221656,0.000000,0.166210,0.000000,0.000000,0.101970,0.846805,0.500000,1.000000,0.500000,1.000000,0.75,0.933013,1.0
1,0.305240,0.148353,0.992553,0.215484,0.000000,0.161612,0.000000,0.000000,0.094208,0.853070,0.629410,0.982963,0.500000,1.000000,0.75,0.933013,1.0
2,0.300254,0.156219,0.997234,0.190493,0.000000,0.169659,0.000000,0.000000,0.084359,0.821742,0.750000,0.933013,0.500000,1.000000,0.75,0.933013,1.0
3,0.303199,0.152471,0.965745,0.170245,0.000000,0.168400,0.000000,0.000000,0.070518,0.829261,0.853553,0.853553,0.500000,1.000000,0.75,0.933013,1.0
4,0.307582,0.142761,0.957447,0.174172,0.000000,0.157670,0.000000,0.000000,0.067150,0.846805,0.933013,0.750000,0.500000,1.000000,0.75,0.933013,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8778,0.854390,0.142699,0.124255,0.125975,0.042631,0.162050,0.028531,0.054453,0.062720,0.757832,0.000000,0.500000,0.900969,0.801938,0.50,1.000000,0.0
8779,0.842244,0.110927,0.130851,0.119141,0.000180,0.131775,0.000052,0.000402,0.024385,0.816103,0.017037,0.629410,0.900969,0.801938,0.50,1.000000,0.0
8780,0.832675,0.087205,0.141915,0.122150,0.000000,0.107303,0.000000,0.000000,0.033575,0.848997,0.066987,0.750000,0.900969,0.801938,0.50,1.000000,0.0
8781,0.646748,0.065757,0.176596,0.140460,0.000000,0.083653,0.000000,0.000000,0.051882,0.869674,0.146447,0.853553,0.900969,0.801938,0.50,1.000000,0.0


In [ ]:
# 5. Guardar el dataset normalizado
df_scaled.to_csv('Dataset_PVPC_Normalizado.csv', index=False)

# 6. GUARDAR LOS ESCALADORES (Muy importante para la fase de predicción)
joblib.dump(scaler_features, 'scaler_features.pkl')
joblib.dump(scaler_target, 'scaler_target.pkl')

print("Proceso completado. Se han generado los siguientes archivos:")
print("- Dataset_PVPC_Normalizado.csv")
print("- scaler_features.pkl")
print("- scaler_target.pkl")

Proceso completado. Se han generado los siguientes archivos:
- Dataset_PVPC_Normalizado.csv
- scaler_features.pkl
- scaler_target.pkl


#Creo la secuencia de tener en cuenta las últimas 24h

In [ ]:
def crear_secuencias(data, n_pasos):
    X, y = [], []
    for i in range(len(data) - n_pasos):
        # Cogemos las filas desde i hasta i + n_pasos
        X.append(data.iloc[i:(i + n_pasos)].values)
        # Cogemos el precio de la fila i + n_pasos (el futuro inmediato)
        y.append(data.iloc[i + n_pasos, 0]) # El precio es la columna 0
    return np.array(X), np.array(y)

# Definimos que el modelo mire las últimas 24 horas
n_pasos = 24
X, y = crear_secuencias(df, n_pasos)

print(f"Forma de X (Entradas): {X.shape}")
# Resultado esperado: (8759, 24, 17) -> 8759 muestras, 24 horas cada una, 17 variables
print(f"Forma de y (Objetivo): {y.shape}")

#USAR ESCALADORES PARA DESNORMALIZAR O NORMALIZAR SIGUIENDO EL MISMO PATRON

In [ ]:
import joblib

# Cargamos los "traductores" que guardamos antes
scaler_features = joblib.load('scaler_features.pkl')
scaler_target = joblib.load('scaler_target.pkl')

In [ ]:
# Datos nuevos (en formato real)
nuevos_datos = [[20.5, 10.0, 15.2, 100.0, 18.5, 50.0, 25.0, 20.0, 60.0, 0.5, 0.8, 0.0, 1.0, 0.5, 0.8, 0]]

# El modelo LSTM no entiende "20.5°C", entiende valores entre 0 y 1
datos_normalizados = scaler_features.transform(nuevos_datos)

# Ahora sí podrías hacer: prediccion = modelo.predict(datos_normalizados)

In [ ]:
# Supongamos que esta es la salida de tu red neuronal
prediccion_red = [[0.4253]]

# Usamos el escalador del target para volver a Euros
precio_real = scaler_target.inverse_transform(prediccion_red)

print(f"El precio real predicho es: {precio_real[0][0]} €/MWh")